Use recipes from the RAW_recipes.csv dataset downloaded from https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_recipes.csv

In [1]:
import json
import pandas as pd
import numpy as np
from wordcloud import WordCloud

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer
import gensim
from gensim import models
from gensim.models import Word2Vec
from gensim.models import CoherenceModel
from tqdm import tqdm
from textblob import TextBlob
import json
import re
import spacy
import matplotlib.pyplot as plt

from multiprocessing import Pool

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%time
recipe_df = pd.read_csv('/content/drive/MyDrive/GCPLP/RAW_recipes.csv')
#recipe_df = pd.read_csv('/content/drive/MyDrive/GCPLP/RAW_recipes_small.csv')

CPU times: user 5.49 s, sys: 517 ms, total: 6.01 s
Wall time: 9.37 s


In [4]:
recipe_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [5]:
recipe_df.shape

(231637, 12)

In [6]:
# drop the columns that are not needed
recipe_df.drop({'id','minutes','contributor_id', 'submitted','tags', 'nutrition', 'n_steps','description', 'n_ingredients'}, axis=1, inplace=True)

In [7]:
recipe_df.head()

,name,steps,ingredients
0,arriba baked winter squash mexican style,"['make a choice and proceed with recipe', 'dep...","['winter squash', 'mexican seasoning', 'mixed ..."
1,a bit different breakfast pizza,"['preheat oven to 425 degrees f', 'press dough...","['prepared pizza crust', 'sausage patty', 'egg..."
2,all in the kitchen chili,"['brown ground beef in large pot', 'add choppe...","['ground beef', 'yellow onions', 'diced tomato..."
3,alouette potatoes,['place potatoes in a large pot of lightly sal...,"['spreadable cheese with garlic and herbs', 'n..."
4,amish tomato ketchup for canning,['mix all ingredients& boil for 2 1 / 2 hours ...,"['tomato juice', 'apple cider vinegar', 'sugar..."


In [8]:
recipe_df.iloc[0][1]

"['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']"

In [9]:
#column for combined direction with numbered steps

recipe_df['combined_steps'] = 5

In [10]:
recipe_df['combined_ingredients'] = 5

In [11]:
recipe_df['combined_recipe'] = 5

In [12]:
import ast

In [13]:
recipe_steps = ast.literal_eval(recipe_df.iloc[0][1])

In [14]:
recipe_steps

['make a choice and proceed with recipe',
 'depending on size of squash , cut into half or fourths',
 'remove seeds',
 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece',
 'season with mexican seasoning mix ii',
 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece',
 'season with sweet mexican spice mix',
 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin',
 'be careful not to burn the squash especially if you opt to use sugar or butter',
 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking',
 'if desired , season with salt']

In [15]:
count_int = int(recipe_df.shape[0])

In [16]:
modified_list = []

for i, value in enumerate(ast.literal_eval(recipe_df.iloc[0][1]), start=1):
    modified_list.append(f'{i}) {value}')



In [17]:
modified_list

['1) make a choice and proceed with recipe',
 '2) depending on size of squash , cut into half or fourths',
 '3) remove seeds',
 '4) for spicy squash , drizzle olive oil or melted butter over each cut squash piece',
 '5) season with mexican seasoning mix ii',
 '6) for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece',
 '7) season with sweet mexican spice mix',
 '8) bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin',
 '9) be careful not to burn the squash especially if you opt to use sugar or butter',
 '10) if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking',
 '11) if desired , season with salt']

In [18]:
for ind in recipe_df.index:
  modified_list = []

  for i, value in enumerate(ast.literal_eval(recipe_df.iloc[ind][1]), start=1):
    modified_list.append(f'{i}) {value}')

  #print (' '.join(modified_list))
  val = ' '.join(modified_list)
  recipe_df.at[ind, 'combined_steps'] = val


In [19]:
recipe_df.iloc[0][3]

'1) make a choice and proceed with recipe 2) depending on size of squash , cut into half or fourths 3) remove seeds 4) for spicy squash , drizzle olive oil or melted butter over each cut squash piece 5) season with mexican seasoning mix ii 6) for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece 7) season with sweet mexican spice mix 8) bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin 9) be careful not to burn the squash especially if you opt to use sugar or butter 10) if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking 11) if desired , season with salt'

In [20]:
recipe_df.head()

,name,steps,ingredients,combined_steps,combined_ingredients,combined_recipe
0,arriba baked winter squash mexican style,"['make a choice and proceed with recipe', 'dep...","['winter squash', 'mexican seasoning', 'mixed ...",1) make a choice and proceed with recipe 2) de...,5,5
1,a bit different breakfast pizza,"['preheat oven to 425 degrees f', 'press dough...","['prepared pizza crust', 'sausage patty', 'egg...",1) preheat oven to 425 degrees f 2) press doug...,5,5
2,all in the kitchen chili,"['brown ground beef in large pot', 'add choppe...","['ground beef', 'yellow onions', 'diced tomato...",1) brown ground beef in large pot 2) add chopp...,5,5
3,alouette potatoes,['place potatoes in a large pot of lightly sal...,"['spreadable cheese with garlic and herbs', 'n...",1) place potatoes in a large pot of lightly sa...,5,5
4,amish tomato ketchup for canning,['mix all ingredients& boil for 2 1 / 2 hours ...,"['tomato juice', 'apple cider vinegar', 'sugar...",1) mix all ingredients& boil for 2 1 / 2 hours...,5,5


In [21]:
recipe_df.iloc[0][2]

"['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']"

In [22]:
ast.literal_eval(recipe_df.iloc[0][2])

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt']

In [23]:
for ind in recipe_df.index:
  modified_list = []

  modified_list.append(ast.literal_eval(recipe_df.iloc[ind][2]))

  #print(modified_list)
  #print (' '.join(modified_list))
  #val = ' '.join(modified_list)
  #recipe_df.at[ind, 'combined_ingredients'] = val
  result = ''.join(', '.join(l) for l in modified_list)
  recipe_df.at[ind, 'combined_ingredients'] = result

In [24]:
recipe_df.columns

Index(['name', 'steps', 'ingredients', 'combined_steps',
       'combined_ingredients', 'combined_recipe'],
      dtype='object')

In [25]:
#Prepare title of food
recipe_df.name = recipe_df.name.str.title()

In [26]:
recipe_df.head()

,name,steps,ingredients,combined_steps,combined_ingredients,combined_recipe
0,Arriba Baked Winter Squash Mexican Style,"['make a choice and proceed with recipe', 'dep...","['winter squash', 'mexican seasoning', 'mixed ...",1) make a choice and proceed with recipe 2) de...,"winter squash, mexican seasoning, mixed spice,...",5
1,A Bit Different Breakfast Pizza,"['preheat oven to 425 degrees f', 'press dough...","['prepared pizza crust', 'sausage patty', 'egg...",1) preheat oven to 425 degrees f 2) press doug...,"prepared pizza crust, sausage patty, eggs, mil...",5
2,All In The Kitchen Chili,"['brown ground beef in large pot', 'add choppe...","['ground beef', 'yellow onions', 'diced tomato...",1) brown ground beef in large pot 2) add chopp...,"ground beef, yellow onions, diced tomatoes, to...",5
3,Alouette Potatoes,['place potatoes in a large pot of lightly sal...,"['spreadable cheese with garlic and herbs', 'n...",1) place potatoes in a large pot of lightly sa...,"spreadable cheese with garlic and herbs, new p...",5
4,Amish Tomato Ketchup For Canning,['mix all ingredients& boil for 2 1 / 2 hours ...,"['tomato juice', 'apple cider vinegar', 'sugar...",1) mix all ingredients& boil for 2 1 / 2 hours...,"tomato juice, apple cider vinegar, sugar, salt...",5


In [27]:
result = recipe_df.iloc[0][0] + ' ' +  'Steps: ' + recipe_df.iloc[0][3] + ' ' +   'Ingredients: ' +recipe_df.iloc[0][4]
result

'Arriba   Baked Winter Squash Mexican Style Steps: 1) make a choice and proceed with recipe 2) depending on size of squash , cut into half or fourths 3) remove seeds 4) for spicy squash , drizzle olive oil or melted butter over each cut squash piece 5) season with mexican seasoning mix ii 6) for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece 7) season with sweet mexican spice mix 8) bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin 9) be careful not to burn the squash especially if you opt to use sugar or butter 10) if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking 11) if desired , season with salt Ingredients: winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt'

In [35]:
for ind in recipe_df.index:
  #modified_list = []

  #modified_list.append(ast.literal_eval(recipe_df.iloc[ind][2]))

  #print(modified_list)
  #print (' '.join(modified_list))
  #val = ' '.join(modified_list)
  #recipe_df.at[ind, 'combined_ingredients'] = val
  #result = ''.join(', '.join(l) for l in modified_list)
  try:
    result = recipe_df.iloc[ind][0] + ' ' +  'Steps: ' + recipe_df.iloc[ind][3] + ' ' +   'Ingredients: ' +recipe_df.iloc[ind][4]
  except Exception as e:
    print(f"Error processing row : {e}")

  #print(recipe_df.iloc[ind][0])
  #print(recipe_df.iloc[ind][3])
  #print(recipe_df.iloc[ind][4])
  recipe_df.at[ind, 'combined_recipe'] = result

Error processing row : unsupported operand type(s) for +: 'float' and 'str'


In [36]:
recipe_df.head()

,name,steps,ingredients,combined_steps,combined_ingredients,combined_recipe
0,Arriba Baked Winter Squash Mexican Style,"['make a choice and proceed with recipe', 'dep...","['winter squash', 'mexican seasoning', 'mixed ...",1) make a choice and proceed with recipe 2) de...,"winter squash, mexican seasoning, mixed spice,...",Arriba Baked Winter Squash Mexican Style Ste...
1,A Bit Different Breakfast Pizza,"['preheat oven to 425 degrees f', 'press dough...","['prepared pizza crust', 'sausage patty', 'egg...",1) preheat oven to 425 degrees f 2) press doug...,"prepared pizza crust, sausage patty, eggs, mil...",A Bit Different Breakfast Pizza Steps: 1) pre...
2,All In The Kitchen Chili,"['brown ground beef in large pot', 'add choppe...","['ground beef', 'yellow onions', 'diced tomato...",1) brown ground beef in large pot 2) add chopp...,"ground beef, yellow onions, diced tomatoes, to...",All In The Kitchen Chili Steps: 1) brown grou...
3,Alouette Potatoes,['place potatoes in a large pot of lightly sal...,"['spreadable cheese with garlic and herbs', 'n...",1) place potatoes in a large pot of lightly sa...,"spreadable cheese with garlic and herbs, new p...",Alouette Potatoes Steps: 1) place potatoes in...
4,Amish Tomato Ketchup For Canning,['mix all ingredients& boil for 2 1 / 2 hours ...,"['tomato juice', 'apple cider vinegar', 'sugar...",1) mix all ingredients& boil for 2 1 / 2 hours...,"tomato juice, apple cider vinegar, sugar, salt...",Amish Tomato Ketchup For Canning Steps: 1) m...


In [37]:
recipe_df = recipe_df.drop('name', axis=1)
recipe_df = recipe_df.drop('steps', axis=1)
recipe_df = recipe_df.drop('ingredients', axis=1)
recipe_df = recipe_df.drop('combined_steps', axis=1)

In [38]:
recipe_df.head()

,combined_ingredients,combined_recipe
0,"winter squash, mexican seasoning, mixed spice,...",Arriba Baked Winter Squash Mexican Style Ste...
1,"prepared pizza crust, sausage patty, eggs, mil...",A Bit Different Breakfast Pizza Steps: 1) pre...
2,"ground beef, yellow onions, diced tomatoes, to...",All In The Kitchen Chili Steps: 1) brown grou...
3,"spreadable cheese with garlic and herbs, new p...",Alouette Potatoes Steps: 1) place potatoes in...
4,"tomato juice, apple cider vinegar, sugar, salt...",Amish Tomato Ketchup For Canning Steps: 1) m...


In [39]:
# save processed data to csv file
recipe_df.to_csv('/content/drive/MyDrive/GCPLP/LLM_recipe_full.csv', index=False)